First we need to extract data from sqlite database.
Database was downloaded from smartphone using android backup.

In [1]:
def get_data_from_file(db_file):
    import sqlite3
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        cur = conn.cursor()
        cur.execute("SELECT * FROM accelerometer_data")
        rows = cur.fetchall()
        conn.close()
        return rows
    except Error as e:
        print(e)
        return None

In [2]:
data = get_data_from_file('../accelerometer_data.db')

In [3]:
import pandas as pd
df = pd.DataFrame(data, columns = ['id', 'coor_x', 'coor_y', 'coor_z', 'timestamp', 'activity'])
df = df.drop(columns = ['id'])
df.head()

,coor_x,coor_y,coor_z,timestamp,activity
0,1.381454,4.313154,8.368932,1575470973260,WALKING
1,0.183156,5.242105,11.680113,1575470973393,WALKING
2,-0.823605,2.134431,8.748413,1575470973527,WALKING
3,0.882263,2.390610,6.970718,1575470973660,WALKING
4,-0.363919,4.357447,7.726089,1575470973795,WALKING


In [4]:
#normalise
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['coor_x', 'coor_y', 'coor_z']] = scaler.fit_transform(df[['coor_x', 'coor_y', 'coor_z']])
x_data = df[['coor_x', 'coor_y', 'coor_z']]

In [6]:
import sys
sys.path.append('../') #the ROCKET project is in parent directory

from rocket_master.code.rocket_functions import generate_kernels, apply_kernels

In [7]:
# generate random kernels
kernels = generate_kernels(input_length = x_data.shape[1], num_kernels = 10_000)
kernels[0][100] #example kernel, in form: weights, lengths, biases, dilations, paddings

array([ 0.73973445, -0.14256561,  0.1948192 , -0.70367441, -0.78927959,
        0.65520331,  0.67094386, -0.12138162, -0.5037996 ,  0.        ,
        0.        ])

In [8]:
x_transformed = apply_kernels(x_data.to_numpy(), kernels)
x_transformed[0]

array([1.        , 0.75877717, 1.        , ..., 0.31798007, 1.        ,
       0.74093289])

In [9]:
from sklearn.model_selection import train_test_split

#split data for train and test dataset
x_train, x_test, y_train, y_test = train_test_split(x_transformed, df[['activity']], test_size=0.1, random_state=42)

In [10]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
classifier.fit(x_train, y_train.to_numpy().ravel())


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

In [11]:
predictions = classifier.predict(x_test)

# print(f"predictions = {', '.join(predictions.astype(str))}")
print(f"accuracy    = {(predictions == y_test['activity']).mean()}")

accuracy    = 0.6601941747572816


In [ ]:
#check when predictions are wrong
test_df = pd.DataFrame(y_test[predictions != y_test['activity']])
test_df['predicted'] = predictions[predictions != y_test['activity']]
test_df

As we can see, the model predictions are pretty good (about 60/70 %).
We should remember, that very small dataset was used (also with some mistakes), as well as the raw data was loaded - we could have group it into eg. 3-seconds frames, but it wouldn't be apropriate for ROCKET.
